# Radar Interpolation Script

This script interpolates radar data to a regular azimuthal grid. This is primarily needed for cases where there is non-uniform or inconsistent azimuths for each PPI. 

In [5]:
import numpy as np
import numpy.ma as ma
import scipy
import datetime
from netCDF4 import Dataset
import glob
from scipy import interpolate
import dlib

import matplotlib.pyplot as plt

In [2]:
az_int = np.arange(0,360,1)

#Cartesian grid parameters
dx_int = 0.1
dy_int = 0.1
dz_int = 0.25
xmin = -30
xmax = 30
ymin = -30
ymax = 30
zmin = 0
zmax = 5


xg = np.arange(xmin,xmax+dx_int,dx_int)
yg = np.arange(ymin,ymax+dy_int,dy_int)
zg = np.arange(zmin,zmax+dz_int,dz_int)
(xgg,ygg) = np.meshgrid(xg,yg)

In [13]:
#Read raxpol function

def read_raxpol(files):
    f = Dataset(files)
    az=np.deg2rad(f['azimuth'][:])
    el=np.deg2rad(f['elevation'][:])
    r=f['range'][:]/1000
    DBZ = f['DBZ'][:]
    VEL = f['VEL'][:]
    VU = f['VU'][:]
    ZDR = f['ZDR'][:]
    RHOHV = f['RHOHV'][:]
    stime = f.getncattr('start_datetime')
    f.close()
    
    #Mask unreasonable values
    mask = (DBZ < -50.0) | (DBZ > 100.0)
    DBZ[mask] = np.NaN
    VEL[mask] = np.NaN
    VU[mask] = np.NaN
    ZDR[mask] = np.NaN
    RHOHV[mask] = np.NaN
    
    raxpol_obj = {
        'az' : az,
        'el' : el,
        'r' : r,
        'DBZ' : DBZ,
        'VEL' : VEL,
        'VU' : VU,
        'ZDR' : ZDR,
        'RHOHV' : RHOHV,
        'dtime' : stime
    }
    return raxpol_obj

#Interpolate data to Cartesian coordinates on PPI (elevation) plane
def save_data():
    new_time = datetime.datetime.strptime(dtime,'%Y-%m-%dT%H:%M:%SZ').strftime('%Y%m%d_%H%M%S')
    fname = path + 'ncf_'+ new_time + '.nc'
    ds = Dataset(fname, 'w',format='NETCDF4')
    
    #Define dimensions
    time = ds.createDimension('time', None)
    x0 = ds.createDimension('x0',Z3d.shape[0])
    y0 = ds.createDimension('y0',Z3d.shape[1])
    z0 = ds.createDimension('z0',Z3d.shape[2])
    
    #Define variables
    x = ds.createVariable('x',np.float32,('x0',))
    x.units = 'km'
    x.axis = 'X'
    x.standard_name = 'projection_x_coordinate'
    y = ds.createVariable('y',np.float32,('y0',))
    y.units = x.units
    y.axis = 'Y'
    y.standard_name = 'projection_y_coordinate'
    z = ds.createVariable('z',np.float32,('z0',))
    z.units = x.units
    z.standard_name = 'projection_z_coordinate'
    z.axis = 'Z'
    
    dbz = ds.createVariable('DBZ',np.float32,('time','z0','y0','x0'))
    dbz.units = 'dBZ'
    dbz.standard_name = 'equivalent_reflectivity_factor'
    
    vr = ds.createVariable('VR',np.float32,('time','z0','y0','x0'))
    vr.units = 'm s-1'
    vr.standard_name = 'radial_velocity_of_scatterers_away_from_instrument'
    
    #Define attributes
    ds.Conventions = 'CF-1.10'
    ds.title = 'RaXPol'
    ds.institution = 'OU' 
    ds.comment = 'Uses Homeyer GridRad approach to gridding data'
    
    #Write Data to variables
    x[:] = xg
    y[:] = yg
    z[:] = zg
    dbz[:,:,:,:] = np.transpose(Z3d[:,:,:,np.newaxis],(3,2,1,0))
    vr[:,:,:,:] = np.transpose(vr3d[:,:,:,np.newaxis],(3,2,1,0))
    
    ds.close()

In [4]:
path = '/Users/dbodine/Documents/Data/test_raxpol/'
fcnt = 0

el_prev = 100
scan_num = 0
vol_num = 0

scans = []
vols = []

flist = sorted(glob.glob(path + '*.nc'))

for files in flist:
    print(files)
    
    raxpol_obj = read_raxpol(files)
    
    az=raxpol_obj['az']
    el=raxpol_obj['el']
    r=raxpol_obj['r']
    
    DBZ = raxpol_obj['DBZ'][:]
    VEL = raxpol_obj['VEL']
    VU = raxpol_obj['VU']
    ZDR = raxpol_obj['ZDR']
    RHOHV = raxpol_obj['RHOHV']
    dtime = raxpol_obj['dtime']
    
    if(el.mean() > el_prev):
        scan_num = scan_num + 1
    elif(fcnt > 0):
        print('Starting new volume')
        scan_num = 1
        vol_num = vol_num + 1
        #Perform 3D interp
        #Perform final 3D interpolation
        (xd3,yd3,zd3) = np.meshgrid(xg,yg,zg)
        Z3d = np.zeros(xd3.shape)
        vr3d = np.zeros(xd3.shape)

        #Interpolate only in height with a loop (probably a faster way to do this, should 
        #try 3D interpolation)
        #Index convention reversed here for saving the file in CF-compliant format
        for i in range(len(xg)):
            for j in range(len(yg)):
                fZ = interpolate.interp1d(np.squeeze(zzint[i,j,:]),np.squeeze(Zint[i,j,:]),kind='linear',bounds_error=False)
                Z3d[i,j,:] = fZ(zg)
                fvr = interpolate.interp1d(np.squeeze(zzint[i,j,:]),np.squeeze(vrint[i,j,:]),kind='linear',bounds_error=False)
                vr3d[i,j,:] = fvr(zg)
        
        #Save Data
        save_data()
        
        #Overwrite old variables
        Zint=np.zeros((xgg.shape[0],ygg.shape[1],10))
        vrint=np.zeros((xgg.shape[0],ygg.shape[1],10))
        zzint=np.zeros((xgg.shape[0],ygg.shape[1],10))
        del xd3, yd3, zd3, Z3d, vr3d
    else:
        print('Starting new volume')
        scan_num = 1
        vol_num = vol_num + 1
        #Perform 3D interp
        #Perform final 3D interpolation
        (xd3,yd3,zd3) = np.meshgrid(xg,yg,zg)
        Z3d = np.zeros(xd3.shape)
        vr3d = np.zeros(xd3.shape)
        
    el_prev = el.mean()
    print('Current ele is: ',el_prev*180/3.1415)
    scans.append(scan_num)
    vols.append(vol_num)
    print(scan_num)
    print(vol_num)   
    
    if(fcnt == 0):
        Zint=np.zeros((xgg.shape[0],ygg.shape[1],10))
        vrint=np.zeros((xgg.shape[0],ygg.shape[1],10))
        zzint=np.zeros((xgg.shape[0],ygg.shape[1],10))
    
    #mask invalid values
    array = np.ma.masked_invalid(DBZ)
    rr,az1 = np.meshgrid(r,az.transpose())
    
    xx1 = rr*np.sin(az1)
    yy1 = rr*np.cos(az1)
    rrg = np.sqrt(xgg*xgg+ygg*ygg)
    zzint[:,:,fcnt] = dlib.bh_calc(rrg,el_prev)
    
    x1 = xx1[~array.mask]
    y1 = yy1[~array.mask]
    newarr = array[~array.mask]
    
    Zint[:,:,fcnt] = interpolate.griddata((x1,y1),newarr.ravel(),
                                         (xgg,ygg),
                                         method='linear')
    
    array = np.ma.masked_invalid(VU)
    x1 = xx1[~array.mask]
    y1 = yy1[~array.mask]
    newarr = array[~array.mask]
    
    vrint[:,:,fcnt] = interpolate.griddata((x1,y1),newarr.ravel(),
                                         (xgg,ygg),
                                         method='linear')
    
    #Increment file count
    fcnt = fcnt + 1
    
    
#Perform final 3D interpolation
#Try actual 3D interpolation
(xd3,yd3,zd3) = np.meshgrid(xg,yg,zg)
Z3d = np.zeros(xd3.shape)
vr3d = np.zeros(xd3.shape)

#Interpolate only in height with a loop (probably a faster way to do this, should 
#try 3D interpolation)
for i in range(len(xg)):
    for j in range(len(yg)):
        fZ = interpolate.interp1d(np.squeeze(zzint[i,j,:]),np.squeeze(Zint[i,j,:]),kind='linear',bounds_error=False)
        Z3d[i,j,:] = fZ(zg)
        fvr = interpolate.interp1d(np.squeeze(zzint[i,j,:]),np.squeeze(vrint[i,j,:]),kind='linear',bounds_error=False)
        vr3d[i,j,:] = fvr(zg)
        
save_data()
plt.figure()
plt.pcolormesh(DBZ,vmin=-10,vmax=70)

/Users/dbodine/Documents/Data/test_raxpol/cfrad.20110524_204835.897_to_20110524_204837.795_RaXpol_SUR.nc
Starting new volume
Current ele is:  3.4245076965780177
1
1
/Users/dbodine/Documents/Data/test_raxpol/cfrad.20110524_204837.800_to_20110524_204839.593_RaXpol_SUR.nc
Current ele is:  5.63004793121219
2
1
/Users/dbodine/Documents/Data/test_raxpol/cfrad.20110524_204839.599_to_20110524_204841.397_RaXpol_SUR.nc
Current ele is:  7.571405366407935
3
1
/Users/dbodine/Documents/Data/test_raxpol/cfrad.20110524_204841.403_to_20110524_204843.289_RaXpol_SUR.nc
Current ele is:  9.543270546548742
4
1
/Users/dbodine/Documents/Data/test_raxpol/cfrad.20110524_204843.295_to_20110524_204845.198_RaXpol_SUR.nc
Current ele is:  11.58224116470511
5
1
/Users/dbodine/Documents/Data/test_raxpol/cfrad.20110524_204845.203_to_20110524_204846.996_RaXpol_SUR.nc
Current ele is:  13.63479182482433
6
1
/Users/dbodine/Documents/Data/test_raxpol/cfrad.20110524_204847.002_to_20110524_204848.795_RaXpol_SUR.nc
Current ele

NameError: name 'datetime' is not defined

In [ ]:
plt.pcolormesh(xgg,ygg,Zint[:,:,1])
plt.colorbar()

In [ ]:
plt.pcolormesh(xgg,ygg,vrint[:,:,5])
plt.colorbar()

In [ ]:
plt.pcolormesh(VU)
plt.colorbar()

In [ ]:
print(f)

In [14]:
dtime = raxpol_obj['dtime']
save_data()